## #1

In [ ]:
!git clone https://github.com/Mongsel8/DPR
%cd /content/DPR

In [ ]:
!pip install .
!python /content/DPR/dpr/data/download_data.py --resource data.retriever.nq-dev
!python /content/DPR/dpr/data/download_data.py --resource data.retriever.nq-train

In [ ]:
%mkdir /content/output_nq
!python /content/DPR/train_dense_encoder.py train_datasets=[nq_train] dev_datasets=[nq_dev] train=biencoder_local output_dir="/content/output_nq"

## #2

In [ ]:
%mkdir /content/drive/MyDrive/output_reader
%mkdir /content/drive/MyDrive/output_retriever

mkdir: cannot create directory ‘/content/drive/MyDrive/output_reader’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/output_retriever’: File exists


In [ ]:
# 시간이 굉장히 오래걸림
!git clone https://github.com/Mongsel8/DPR

%cd /content/DPR
!pip install .
!python /content/DPR/dpr/data/download_data.py --resource data.retriever.nq-dev
!python /content/DPR/dpr/data/download_data.py --resource data.retriever.nq-train
!python /content/DPR/train_dense_encoder.py train_datasets=[nq_train] dev_datasets=[nq_dev] train=biencoder_local output_dir="/content/drive/MyDrive/output_retriever" 

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[2023-02-07 08:20:26,897][root][INFO] - Epoch: 0: Step: 31702/58880, loss=0.000000, lr=0.000020
[2023-02-07 08:20:27,129][root][INFO] - Epoch: 0: Step: 31703/58880, loss=0.000000, lr=0.000020
[2023-02-07 08:20:27,356][root][INFO] - Epoch: 0: Step: 31704/58880, loss=0.000000, lr=0.000020
[2023-02-07 08:20:27,584][root][INFO] - Epoch: 0: Step: 31705/58880, loss=0.000010, lr=0.000020
[2023-02-07 08:20:27,819][root][INFO] - Epoch: 0: Step: 31706/58880, loss=0.000000, lr=0.000020
[2023-02-07 08:20:28,049][root][INFO] - Epoch: 0: Step: 31707/58880, loss=0.000000, lr=0.000020
[2023-02-07 08:20:28,281][root][INFO] - Epoch: 0: Step: 31708/58880, loss=0.515424, lr=0.000020
[2023-02-07 08:20:28,520][root][INFO] - Epoch: 0: Step: 31709/58880, loss=4.562493, lr=0.000020
[2023-02-07 08:20:28,751][root][INFO] - Epoch: 0: Step: 31710/58880, loss=0.000000, lr=0.000020
[2023-02-07 08:20:28,980][root][INFO] - Epoch: 0: Step: 31711/58880, loss=0.000000, lr=0.000020
[202

## #3

In [ ]:
""" 파일명
encoder train default yaml 

# @package _group_

nq_train:
  _target_: dpr.data.biencoder_data.JsonQADataset
  file: data.retriever.nq-train

nq_train_hn1:
  _target_: dpr.data.biencoder_data.JsonQADataset
  file: data.retriever.nq-adv-hn-train

nq_dev:
  _target_: dpr.data.biencoder_data.JsonQADataset
  file: data.retriever.nq-dev

trivia_train:
  _target_: dpr.data.biencoder_data.JsonQADataset
  file: data.retriever.trivia-train

trivia_dev:
  _target_: dpr.data.biencoder_data.JsonQADataset
  file: data.retriever.trivia-dev

squad1_train:
  _target_: dpr.data.biencoder_data.JsonQADataset
  file: data.retriever.squad1-train

squad1_dev:
  _target_: dpr.data.biencoder_data.JsonQADataset
  file: data.retriever.squad1-dev

webq_train:
  _target_: dpr.data.biencoder_data.JsonQADataset
  file: data.retriever.webq-train

webq_dev:
  _target_: dpr.data.biencoder_data.JsonQADataset
  file: data.retriever.webq-dev

curatedtrec_train:
  _target_: dpr.data.biencoder_data.JsonQADataset
  file: data.retriever.curatedtrec-train

curatedtrec_dev:
  _target_: dpr.data.biencoder_data.JsonQADataset
  file: data.retriever.curatedtrec-dev

korquad_train:
  _target_: dpr.data.biencoder_data.JsonQADataset
  file: data.retriever.korquad-train
  
korquad_dev:
  _target_: dpr.data.biencoder_data.JsonQADataset
  file: data.retriever.korquad-dev
"""

In [ ]:
""" 파일명
hf bert yaml

# @package _group_

# model type. One of [hf_bert, pytext_bert, fairseq_roberta]
encoder_model_type: hf_bert

# HuggingFace's config name for model initialization
pretrained_model_cfg: beomi/kcbert-base

# Some encoders need to be initialized from a file
pretrained_file:

# Extra linear layer on top of standard bert/roberta encoder
projection_dim: 0

# Max length of the encoder input sequence
sequence_length: 256

dropout: 0.1

# whether to fix (don't update) context encoder during training or not
fix_ctx_encoder: False

# if False, the model won't load pre-trained BERT weights
pretrained: True
"""

In [ ]:
#!/usr/bin/env python3
# Copyright (c) Facebook, Inc. and its affiliates.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.

"""
 Command line tool to download various preprocessed data sources & checkpoints for DPR
"""

import argparse
import gzip
import logging
import os
import pathlib
import wget

from typing import Tuple

logger = logging.getLogger(__name__)

# TODO: move to hydra config group

NQ_LICENSE_FILES = [
    "https://dl.fbaipublicfiles.com/dpr/nq_license/LICENSE", "https://dl.fbaipublicfiles.com/dpr/nq_license/README",
    ]

RESOURCES_MAP = {
    "data.wikipedia_split.psgs_w100": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/wikipedia_split/psgs_w100.tsv.gz",
        "original_ext": ".tsv",
        "compressed": True,
        "desc": "Entire wikipedia passages set obtain by splitting all pages into 100-word segments (no overlap)",
    },
    "data.retriever.nq-dev": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/data/retriever/biencoder-nq-dev.json.gz",
        "original_ext": ".json",
        "compressed": True,
        "desc": "NQ dev subset with passages pools for the Retriever train time validation",
        "license_files": NQ_LICENSE_FILES,
    },
    "data.retriever.nq-train": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/data/retriever/biencoder-nq-train.json.gz",
        "original_ext": ".json",
        "compressed": True,
        "desc": "NQ train subset with passages pools for the Retriever training",
        "license_files": NQ_LICENSE_FILES,
    },
    "data.retriever.nq-adv-hn-train": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/data/retriever/biencoder-nq-adv-hn-train.json.gz",
        "original_ext": ".json",
        "compressed": True,
        "desc": "NQ train subset with hard negative passages mined using the baseline DPR NQ encoders & wikipedia index",
        "license_files": NQ_LICENSE_FILES,
    },
    "data.retriever.trivia-dev": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/data/retriever/biencoder-trivia-dev.json.gz",
        "original_ext": ".json",
        "compressed": True,
        "desc": "TriviaQA dev subset with passages pools for the Retriever train time validation",
    },
    "data.retriever.trivia-train": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/data/retriever/biencoder-trivia-train.json.gz",
        "original_ext": ".json",
        "compressed": True,
        "desc": "TriviaQA train subset with passages pools for the Retriever training",
    },
    "data.retriever.squad1-train": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/data/retriever/biencoder-squad1-train.json.gz",
        "original_ext": ".json",
        "compressed": True,
        "desc": "SQUAD 1.1 train subset with passages pools for the Retriever training",
    },
    "data.retriever.squad1-dev": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/data/retriever/biencoder-squad1-dev.json.gz",
        "original_ext": ".json",
        "compressed": True,
        "desc": "SQUAD 1.1 dev subset with passages pools for the Retriever train time validation",
    },
    "data.retriever.webq-train": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/data/retriever/biencoder-webquestions-train.json.gz",
        "original_ext": ".json",
        "compressed": True,
        "desc": "WebQuestions dev subset with passages pools for the Retriever train time validation",
    },
    "data.retriever.webq-dev": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/data/retriever/biencoder-webquestions-dev.json.gz",
        "original_ext": ".json",
        "compressed": True,
        "desc": "WebQuestions dev subset with passages pools for the Retriever train time validation",
    },
    "data.retriever.curatedtrec-train": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/data/retriever/biencoder-curatedtrec-train.json.gz",
        "original_ext": ".json",
        "compressed": True,
        "desc": "CuratedTrec dev subset with passages pools for the Retriever train time validation",
    },
    "data.retriever.curatedtrec-dev": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/data/retriever/biencoder-curatedtrec-dev.json.gz",
        "original_ext": ".json",
        "compressed": True,
        "desc": "CuratedTrec dev subset with passages pools for the Retriever train time validation",
    },
    "data.retriever.qas.nq-dev": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/data/retriever/nq-dev.qa.csv",
        "original_ext": ".csv",
        "compressed": False,
        "desc": "NQ dev subset for Retriever validation and IR results generation",
        "license_files": NQ_LICENSE_FILES,
    },
    "data.retriever.qas.nq-test": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/data/retriever/nq-test.qa.csv",
        "original_ext": ".csv",
        "compressed": False,
        "desc": "NQ test subset for Retriever validation and IR results generation",
        "license_files": NQ_LICENSE_FILES,
    },
    "data.retriever.qas.nq-train": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/data/retriever/nq-train.qa.csv",
        "original_ext": ".csv",
        "compressed": False,
        "desc": "NQ train subset for Retriever validation and IR results generation",
        "license_files": NQ_LICENSE_FILES,
    },
    #
    "data.retriever.qas.trivia-dev": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/data/retriever/trivia-dev.qa.csv.gz",
        "original_ext": ".csv",
        "compressed": True,
        "desc": "Trivia dev subset for Retriever validation and IR results generation",
    },
    "data.retriever.qas.trivia-test": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/data/retriever/trivia-test.qa.csv.gz",
        "original_ext": ".csv",
        "compressed": True,
        "desc": "Trivia test subset for Retriever validation and IR results generation",
    },
    "data.retriever.qas.trivia-train": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/data/retriever/trivia-train.qa.csv.gz",
        "original_ext": ".csv",
        "compressed": True,
        "desc": "Trivia train subset for Retriever validation and IR results generation",
    },
    "data.retriever.qas.squad1-test": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/data/retriever/squad1-test.qa.csv",
        "original_ext": ".csv",
        "compressed": False,
        "desc": "Trivia test subset for Retriever validation and IR results generation",
    },
    "data.retriever.qas.webq-test": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/data/retriever/webquestions-test.qa.csv",
        "original_ext": ".csv",
        "compressed": False,
        "desc": "WebQuestions test subset for Retriever validation and IR results generation",
    },
    "data.retriever.qas.curatedtrec-test": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/data/retriever/curatedtrec-test.qa.csv",
        "original_ext": ".csv",
        "compressed": False,
        "desc": "CuratedTrec test subset for Retriever validation and IR results generation",
    },
    "data.gold_passages_info.nq_train": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/data/nq_gold_info/nq-train_gold_info.json.gz",
        "original_ext": ".json",
        "compressed": True,
        "desc": "Original NQ (our train subset) gold positive passages and alternative question tokenization",
        "license_files": NQ_LICENSE_FILES,
    },
    "data.gold_passages_info.nq_dev": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/data/nq_gold_info/nq-dev_gold_info.json.gz",
        "original_ext": ".json",
        "compressed": True,
        "desc": "Original NQ (our dev subset) gold positive passages and alternative question tokenization",
        "license_files": NQ_LICENSE_FILES,
    },
    "data.gold_passages_info.nq_test": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/data/nq_gold_info/nq-test_gold_info.json.gz",
        "original_ext": ".json",
        "compressed": True,
        "desc": "Original NQ (our test, original dev subset) gold positive passages and alternative question "
        "tokenization",
        "license_files": NQ_LICENSE_FILES,
    },
    "pretrained.fairseq.roberta-base.dict": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/pretrained/fairseq/roberta/dict.txt",
        "original_ext": ".txt",
        "compressed": False,
        "desc": "Dictionary for pretrained fairseq roberta model",
    },
    "pretrained.fairseq.roberta-base.model": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/pretrained/fairseq/roberta/model.pt",
        "original_ext": ".pt",
        "compressed": False,
        "desc": "Weights for pretrained fairseq roberta base model",
    },
    "pretrained.pytext.bert-base.model": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/pretrained/pytext/bert/bert-base-uncased.pt",
        "original_ext": ".pt",
        "compressed": False,
        "desc": "Weights for pretrained pytext bert base model",
    },
    "data.retriever_results.nq.single.wikipedia_passages": {
        "s3_url": ["https://dl.fbaipublicfiles.com/dpr/data/wiki_encoded/single/nq/wiki_passages_{}".format(i)
            for i in range(50)
        ],
        "original_ext": ".pkl",
        "compressed": False,
        "desc": "Encoded wikipedia files using a biencoder checkpoint("
        "checkpoint.retriever.single.nq.bert-base-encoder) trained on NQ dataset ",
    },
    "data.retriever_results.nq.single-adv-hn.wikipedia_passages": {
        "s3_url": ["https://dl.fbaipublicfiles.com/dpr/data/wiki_encoded/single-adv-hn/nq/wiki_passages_{}".format(i)
            for i in range(50)
        ],
        "original_ext": ".pkl",
        "compressed": False,
        "desc": "Encoded wikipedia files using a biencoder checkpoint("
        "checkpoint.retriever.single-adv-hn.nq.bert-base-encoder) trained on NQ dataset + adversarial hard negatives",
    },
    "data.retriever_results.nq.single.test": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/data/retriever_results/single/nq-test.json.gz",
        "original_ext": ".json",
        "compressed": True,
        "desc": "Retrieval results of NQ test dataset for the encoder trained on NQ",
        "license_files": NQ_LICENSE_FILES,
    },
    "data.retriever_results.nq.single.dev": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/data/retriever_results/single/nq-dev.json.gz",
        "original_ext": ".json",
        "compressed": True,
        "desc": "Retrieval results of NQ dev dataset for the encoder trained on NQ",
        "license_files": NQ_LICENSE_FILES,
    },
    "data.retriever_results.nq.single.train": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/data/retriever_results/single/nq-train.json.gz",
        "original_ext": ".json",
        "compressed": True,
        "desc": "Retrieval results of NQ train dataset for the encoder trained on NQ",
        "license_files": NQ_LICENSE_FILES,
    },
    "data.retriever_results.nq.single-adv-hn.test": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/data/retriever_results/single-adv-hn/nq-test.json.gz",
        "original_ext": ".json",
        "compressed": True,
        "desc": "Retrieval results of NQ test dataset for the encoder trained on NQ + adversarial hard negatives",
        "license_files": NQ_LICENSE_FILES,
    },
    "checkpoint.retriever.single.nq.bert-base-encoder": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/checkpoint/retriever/single/nq/hf_bert_base.cp",
        "original_ext": ".cp",
        "compressed": False,
        "desc": "Biencoder weights trained on NQ data and HF bert-base-uncased model",
    },
    "checkpoint.retriever.multiset.bert-base-encoder": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/checkpoint/retriver/multiset/hf_bert_base.cp",
        "original_ext": ".cp",
        "compressed": False,
        "desc": "Biencoder weights trained on multi set data and HF bert-base-uncased model",
    },
    "checkpoint.retriever.single-adv-hn.nq.bert-base-encoder": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/checkpoint/retriver/single-adv-hn/nq/hf_bert_base.cp",
        "original_ext": ".cp",
        "compressed": False,
        "desc": "Biencoder weights trained on the original DPR NQ data combined with adversarial hard negatives (See data.retriever.nq-adv-hn-train resource). "
        "The model is HF bert-base-uncased",
    },
    "data.reader.nq.single.train": {
        "s3_url": ["https://dl.fbaipublicfiles.com/dpr/data/reader/nq/single/train.{}.pkl".format(i) for i in range(8)],
        "original_ext": ".pkl",
        "compressed": False,
        "desc": "Reader model NQ train dataset input data preprocessed from retriever results (also trained on NQ)",
        "license_files": NQ_LICENSE_FILES,
    },
    "data.reader.nq.single.dev": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/data/reader/nq/single/dev.0.pkl",
        "original_ext": ".pkl",
        "compressed": False,
        "desc": "Reader model NQ dev dataset input data preprocessed from retriever results (also trained on NQ)",
        "license_files": NQ_LICENSE_FILES,
    },
    "data.reader.nq.single.test": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/data/reader/nq/single/test.0.pkl",
        "original_ext": ".pkl",
        "compressed": False,
        "desc": "Reader model NQ test dataset input data preprocessed from retriever results (also trained on NQ)",
        "license_files": NQ_LICENSE_FILES,
    },
    "data.reader.trivia.multi-hybrid.train": {
        "s3_url": [
            "https://dl.fbaipublicfiles.com/dpr/data/reader/trivia/multi-hybrid/train.{}.pkl".format(i)
            for i in range(8)
        ],
        "original_ext": ".pkl",
        "compressed": False,
        "desc": "Reader model Trivia train dataset input data preprocessed from hybrid retriever results "
        "(where dense part is trained on multiset)",
    },
    "data.reader.trivia.multi-hybrid.dev": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/data/reader/trivia/multi-hybrid/dev.0.pkl",
        "original_ext": ".pkl",
        "compressed": False,
        "desc": "Reader model Trivia dev dataset input data preprocessed from hybrid retriever results "
        "(where dense part is trained on multiset)",
    },
    "data.reader.trivia.multi-hybrid.test": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/data/reader/trivia/multi-hybrid/test.0.pkl",
        "original_ext": ".pkl",
        "compressed": False,
        "desc": "Reader model Trivia test dataset input data preprocessed from hybrid retriever results "
        "(where dense part is trained on multiset)",
    },
    "checkpoint.reader.nq-single.hf-bert-base": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/checkpoint/reader/nq-single/hf_bert_base.cp",
        "original_ext": ".cp",
        "compressed": False,
        "desc": "Reader weights trained on NQ-single retriever results and HF bert-base-uncased model",
    },
    "checkpoint.reader.nq-trivia-hybrid.hf-bert-base": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/checkpoint/reader/nq-trivia-hybrid/hf_bert_base.cp",
        "original_ext": ".cp",
        "compressed": False,
        "desc": "Reader weights trained on Trivia multi hybrid retriever results and HF bert-base-uncased model",
    },
    # extra checkpoints for EfficientQA competition
    "checkpoint.reader.nq-single-subset.hf-bert-base": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/checkpoint/reader/nq-single-seen_only/hf_bert_base.cp",
        "original_ext": ".cp",
        "compressed": False,
        "desc": "Reader weights trained on NQ-single retriever results and HF bert-base-uncased model, when only Wikipedia pages seen during training are considered",
    },
    "checkpoint.reader.nq-tfidf.hf-bert-base": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/checkpoint/reader/nq-drqa/hf_bert_base.cp",
        "original_ext": ".cp",
        "compressed": False,
        "desc": "Reader weights trained on TFIDF results and HF bert-base-uncased model",
    },
    "checkpoint.reader.nq-tfidf-subset.hf-bert-base": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/checkpoint/reader/nq-drqa-seen_only/hf_bert_base.cp",
        "original_ext": ".cp",
        "compressed": False,
        "desc": "Reader weights trained on TFIDF results and HF bert-base-uncased model, when only Wikipedia pages seen during training are considered",
    },
    # retrieval indexes
    "indexes.single.nq.full.index": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/checkpoint/indexes/single/nq/full.index.dpr",
        "original_ext": ".dpr",
        "compressed": False,
        "desc": "DPR index on NQ-single retriever",
    },
    "indexes.single.nq.full.index_meta": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/checkpoint/indexes/single/nq/full.index_meta.dpr",
        "original_ext": ".dpr",
        "compressed": False,
        "desc": "DPR index on NQ-single retriever (metadata)",
    },
    "indexes.single.nq.subset.index": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/checkpoint/indexes/single/nq/seen_only.index.dpr",
        "original_ext": ".dpr",
        "compressed": False,
        "desc": "DPR index on NQ-single retriever when only Wikipedia pages seen during training are considered",
    },
    "indexes.single.nq.subset.index_meta": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/checkpoint/indexes/single/nq/seen_only.index_meta.dpr",
        "original_ext": ".dpr",
        "compressed": False,
        "desc": "DPR index on NQ-single retriever when only Wikipedia pages seen during training are considered (metadata)",
    },
    "indexes.tfidf.nq.full": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/checkpoint/indexes/drqa/nq/full-tfidf.npz",
        "original_ext": ".npz",
        "compressed": False,
        "desc": "TFIDF index",
    },
    "indexes.tfidf.nq.subset": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/checkpoint/indexes/drqa/nq/seen_only-tfidf.npz",
        "original_ext": ".npz",
        "compressed": False,
        "desc": "TFIDF index when only Wikipedia pages seen during training are considered",
    },
    # Universal retriever project data
    "data.wikipedia_split.psgs_w100": {
        "s3_url": "https://dl.fbaipublicfiles.com/dpr/wikipedia_split/psgs_w100.tsv.gz",
        "original_ext": ".tsv",
        "compressed": True,
        "desc": "Entire wikipedia passages set obtain by splitting all pages into 100-word segments (no overlap)",
    },
    "data.retriever.korquad-train": {
        "s3_url": "https://drive.google.com/uc?export=download&id=1Duf17wfI6Ude-ZKfNFOu1d8VqnV0Mv1E",
        "original_ext": ".json",
        "compressed": False,
        "desc": "korsquad",
    },
    "data.retriever.korquad-dev": {
        "s3_url": "https://drive.google.com/uc?export=download&id=1usKQiHF-HXsFM1q75BVwgf2vWuNX5vGT",
        "original_ext": ".json",
        "compressed": False,
        "desc": "korquad",
    },
}


def unpack(gzip_file: str, out_file: str):
    
logger.info
("Uncompressing %s", gzip_file)
    input = gzip.GzipFile(gzip_file, "rb")
    s = 
input.read
()
    input.close()
    output = open(out_file, "wb")
    output.write(s)
    output.close()
    
logger.info
(" Saved to %s", out_file)


def download_resource(
    s3_url: str, original_ext: str, compressed: bool, resource_key: str, out_dir: str
) -> Tuple[str, str]:
    
logger.info
("Requested resource from %s", s3_url)
    path_names = resource_key.split(".")

    if out_dir:
        root_dir = out_dir
    else:
        # since hydra overrides the location for the 'current dir' for every run and we don't want to duplicate
        # resources multiple times, remove the current folder's volatile part
        root_dir = os.path.abspath("./")
        if "/outputs/" in root_dir:
            root_dir = root_dir[: root_dir.index("/outputs/")]

    
logger.info
("Download root_dir %s", root_dir)

    save_root = os.path.join(root_dir, "downloads", *path_names[:-1])  # last segment is for file name

    pathlib.Path(save_root).mkdir(parents=True, exist_ok=True)

    local_file_uncompressed = os.path.abspath(os.path.join(save_root, path_names[-1] + original_ext))
    
logger.info
("File to be downloaded as %s", local_file_uncompressed)

    if os.path.exists(local_file_uncompressed):
        
logger.info
("File already exist %s", local_file_uncompressed)
        return save_root, local_file_uncompressed

    local_file = os.path.abspath(os.path.join(save_root, path_names[-1] + (".tmp" if compressed else original_ext)))

    
wget.download
(s3_url, out=local_file)

    
logger.info
("Downloaded to %s", local_file)

    if compressed:
        uncompressed_file = os.path.join(save_root, path_names[-1] + original_ext)
        unpack(local_file, uncompressed_file)
        os.remove(local_file)
        local_file = uncompressed_file
    return save_root, local_file


def download_file(s3_url: str, out_dir: str, file_name: str):
    
logger.info("Loading from %s", s3_url)
    local_file = os.path.join(out_dir, file_name)

    if os.path.exists(local_file):
        
logger.info("File already exist %s", local_file)
        return wget.download(s3_url, out=local_file)
    
logger.info("Downloaded to %s", local_file)


def download(resource_key: str, out_dir: str = None):
    if resource_key not in RESOURCES_MAP:
        # match by prefix
        resources = [k for k in RESOURCES_MAP.keys() if k.startswith(resource_key)]
        print(resources)
        
logger.info("matched by prefix resources: %s", resources)
        if resources:
            for key in resources:
                download(key, out_dir)
        else:
            
logger.info("no resources found for specified key")
        return []
    download_info = RESOURCES_MAP[resource_key]

    s3_url = download_info["s3_url"]

    save_root_dir = None
    data_files = []
    if isinstance(s3_url, list):
        for i, url in enumerate(s3_url):
            save_root_dir, local_file = download_resource(
                url,
                download_info["original_ext"],
                download_info["compressed"],
                "{}_{}".format(resource_key, i),
                out_dir,
            )
            data_files.append(local_file)
    else:
        save_root_dir, local_file = download_resource(
            s3_url,
            download_info["original_ext"],
            download_info["compressed"],
            resource_key,
            out_dir,
        )
        data_files.append(local_file)

    license_files = download_info.get("license_files", None)

    if license_files:
        download_file(license_files[0], save_root_dir, "LICENSE")
        download_file(license_files[1], save_root_dir, "README")
    return data_files


def main():
    parser = argparse.ArgumentParser()

    parser.add_argument(
        "--output_dir",
        default="./",
        type=str,
        help="The output directory to download file",
    )
    parser.add_argument(
        "--resource",
        type=str,
        help="Resource name. See RESOURCES_MAP for all possible values",
    )
    args = parser.parse_args()
    if args.resource:
        download(args.resource, args.output_dir)
    else:
        logger.warning("Please specify resource value. Possible options are")
        for k, v in RESOURCES_MAP.items():
            logger.warning("Resource key=%s  :  %s", k, v["desc"])


if __name__ == "__main__":
    main() 

In [ ]:
# 역색인
import math

def BM25(doc, query, k1=1.5, b=0.75, avg_doc_len=0):
    score = 0
    for q in query:
        if q in doc:
            freq = doc[q]
            idf = math.log(len(documents) / df[q])
            numerator = freq * (k1 + 1)
            denominator = freq + k1 * (1 - b + b * (len(doc) / avg_doc_len))
            score += idf * numerator / denominator
    return score

# example usage
documents = [{"this": 1, "is": 1, "a": 1, "sample": 1, "document": 1},
             {"this": 1, "is": 1, "another": 1, "sample": 1, "document": 1}]
df = {"this": 2, "is": 2, "a": 1, "sample": 2, "document": 2, "another": 1}
query = ["this", "sample", "document"] 